# Importing model and libraries

In [1]:
from datasets import load_dataset
#from transformers import LlamaForCausalLM, LlamaTokenizer
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM



/root/llama-distilation/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

In [5]:
#wiki = load_dataset('wikipedia', '20200501.en')

#os.environ["CUDA_LAUNCH_BLOCKING"] ="1" 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").to(device)


#tokenizer = LlamaTokenizer.from_pretrained("./output")
#model = LlamaForCausalLM.from_pretrained("./output",torch_dtype=torch.float16 ).to(device)
#model = LlamaForCausalLM.from_pretrained("./output", )


#from transformers import LlamaConfig

# Initializing a LLaMA llama 2-7b style configuration
#configuration = LlamaConfig(hidden_size=1024,intermediate_size=2752,num_hidden_layers=8,num_attention_heads=8,model_type='llama-2-7b',torch_dtype=torch.float16)

# Initializing a model from the llama 2-7b style configuration
#distiled_model = LlamaForCausalLM(configuration).to(device)
#distiled_model = LlamaForCausalLM(configuration)

# Accessing the model configuration
#configuration = distiled_model.config

from transformers import GPTNeoXConfig, GPTNeoXModel

# Initializing a GPTNeoX gpt-neox-20b style configuration
#configuration = GPTNeoXConfig(hidden_size=)

#distiled_model = GPTNeoXModel(configuration)

configuration = model.config
 
print(configuration)

AttributeError: module 'torch' has no attribute 'frombuffer'

In [4]:
tokenizer.pad_token = tokenizer.eos_token

# Pre Processing

In [5]:
data= wiki['train']

In [6]:
import re

def remove_special_characters(text):
    # Remove special characters using regex
    cleaned_text = re.sub(r'[^a-zA-Z0-9.\s]', '', text)
    return cleaned_text

def remove_urls(text):
    # Remove URLs using regex
    cleaned_text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    return cleaned_text

def pre_process(text):

    pre_processed = text.lower()  
    pre_processed = remove_urls(pre_processed)
    pre_processed = remove_special_characters(pre_processed)
    return pre_processed.strip()


In [7]:
class LLMDistillationDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, tokenizer, max_len):
        self.inputs = inputs
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __getitem__(self, index):
        input_seq = self.inputs[index]
        # Tokenize input sequence using LLama tokenizer
        inputs = self.tokenizer(input_seq, return_tensors='pt', max_length=self.max_len, padding='max_length', truncation=True)
        # Return tokenized inputs and true label as dataset element
        return inputs
    def __len__(self):
        return len(self.inputs)

In [8]:
# Create dataset from data
train_dataset = LLMDistillationDataset(data['text'][:100], tokenizer, max_len=256)


In [9]:
# Create data loaders for training and testing
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training

Initialzing the distiled model

In [11]:
attention_mask = torch.ones_like(train_dataset[0].input_ids) 
generated = model.generate(train_dataset[0].input_ids,attention_mask=attention_mask )

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/root/llama-distilation/env/lib/python3.8/site-packages/transformers/generation/utils.py:1354: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/llama-distilation/env/lib/python3.8/site-packages/transformers/generation/utils.py:1362: UserWarning: Input length of input_ids is 256, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [12]:
#tokenizer.batch_decode(generated, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [13]:
#data[0]['text']

In [10]:
def distillation_loss(student_output, teacher_output):
    student_softmax = torch.softmax(student_output, dim=-1)
    kl_div = torch.sum(torch.square(teacher_output - student_softmax), axis=1)
    return torch.sum(kl_div + 0.1 * (student_softmax - torch.zeros_like(student_softmax)).pow(2), axis=1)

In [13]:

optimizer = torch.optim.AdamW(distiled_model.parameters())
# Train the student model using LLM Distillation
num_epochs = 5
for epoch in range(num_epochs):
    for batch in train_loader:
        # Get input sequences and true labels for current batch
        inputs = batch
        loss_arr = []
        #print(inputs.keys())
        
        optimizer.zero_grad()
        for i in range(10):
            input_ids = inputs['input_ids'][i].to(device)
            #print(f"input ids: {inputs['input_ids'][i]}")
            student_outputs = distiled_model(input_ids)
            teacher_outputs = model(input_ids)
            #print(f'student_outputs: {student_outputs.logits} teacer otuputs: {teacher_outputs.logits}')
            loss = distillation_loss(student_outputs.logits, teacher_outputs.logits)
            #print(f"Current Loss: {loss}")
            loss_arr.append(loss)
        aggregated_loss = torch.stack(loss_arr).mean()
        print(f"Current Loss: {aggregated_loss} ")
        aggregated_loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 6; 23.70 GiB total capacity; 19.39 GiB already allocated; 5.56 MiB free; 19.60 GiB reserved in total by PyTorch)

In [68]:
student_outputs.logits

tensor([[[-0.6310,  0.1586, -0.0924,  ...,  0.0126, -0.8826,  1.0232],
         [-0.2628, -0.0744, -0.1247,  ...,  0.6144, -0.6016,  1.4153],
         [-0.3615, -0.4934, -0.2101,  ...,  0.7223, -0.1389,  1.3991],
         ...,
         [-0.4389,  0.2146, -0.0584,  ..., -0.2479,  1.7112,  0.6876],
         [-0.4351,  0.2258, -0.0608,  ..., -0.2515,  1.7182,  0.6870],
         [-0.4330,  0.2344, -0.0602,  ..., -0.2539,  1.7261,  0.6876]]],
       grad_fn=<UnsafeViewBackward0>)

In [57]:
optimizer = torch.optim.AdamW(distiled_model.parameters())
# Train the student model using LLM Distillation
num_epochs = 5
for epoch in range(num_epochs):
    for batch in train_loader:
        # Get input sequences and true labels for current batch
        inputs = batch
        print(inputs)
        student_outputs = distiled_model(**inputs)
        teacher_outputs = model(**inputs)
        loss = distillation_loss(student_outputs, teacher_outputs)
        optimizer.zero_grad()
        loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

{'input_ids': tensor([[[    1,   450,  4908,  ...,   278,  4642,  4908]],

        [[    1,   450, 21113,  ...,     2,     2,     2]],

        [[    1,   317,  4125,  ...,  5275,    13,    13]],

        ...,

        [[    1,  4644,  3122,  ...,   653, 10719, 15942]],

        [[    1, 10558, 19116,  ...,   278,  5882,   260]],

        [[    1,  9548,   375,  ...,   263, 16151,   393]]]), 'attention_mask': tensor([[[1, 1, 1,  ..., 1, 1, 1]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 1, 1, 1]],

        ...,

        [[1, 1, 1,  ..., 1, 1, 1]],

        [[1, 1, 1,  ..., 1, 1, 1]],

        [[1, 1, 1,  ..., 1, 1, 1]]])}


ValueError: too many values to unpack (expected 2)

In [14]:
inputs['attention_mask']

tensor([[[1, 1, 1,  ..., 1, 1, 1]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 1, 1, 1]],

        ...,

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 1, 1, 1]],

        [[1, 1, 1,  ..., 0, 0, 0]]])